In [1]:
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
import pandas as pd
import numpy as np
import string
from model import new_model
import time

2023-01-15 06:21:24.445194: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-15 06:21:36.918142: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-01-15 06:21:53.798047: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-01-15 06:21:53.798171: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

In [2]:
pretrain_df = pd.read_csv('PreTrain.csv')
finetune_df = pd.read_csv('FineTune.csv')
pretrain_df = pretrain_df['content'].squeeze()
finetune_df = finetune_df['content'].squeeze()

In [3]:
print(pretrain_df.head())
print(finetune_df.head())

0    is mahine buffalo ny me konse concerts aarahe hai
1           is weekend naperville mei free ice skating
2            kya el paso me kabhi tornadoes aa rahe he
3    holidays ke liye wilkes barre me karne ke liye...
4                        latest coldplay song ko bajao
Name: content, dtype: object
0    tumlog karlena baad mein
1                rehne do fir
2       mai kal raat ko jaara
3             mai kal jaaraha
4          waha kya karra hai
Name: content, dtype: object


In [4]:
full_df = pd.concat([pretrain_df, finetune_df])
print(full_df.head())

0    is mahine buffalo ny me konse concerts aarahe hai
1           is weekend naperville mei free ice skating
2            kya el paso me kabhi tornadoes aa rahe he
3    holidays ke liye wilkes barre me karne ke liye...
4                        latest coldplay song ko bajao
Name: content, dtype: object


In [5]:
tokenizer = Tokenizer(lower = False)
tokenizer.fit_on_texts(full_df)
vocab = tokenizer.word_index
idx2word = {u: v for v, u in vocab.items()}
vocab_size = len(vocab) + 1

In [6]:
sequences = tokenizer.texts_to_sequences(pretrain_df)
sequences = tf.keras.utils.pad_sequences(sequences)

In [7]:
X, y = sequences[:, :-1], sequences[:, -1]

In [8]:
y = tf.keras.utils.to_categorical(y, vocab_size - 1)

In [9]:
model = new_model(vocab_size = vocab_size, seq_length = X.shape[1])
print(model.summary())

2023-01-15 06:22:56.306126: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-15 06:23:12.740536: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 7425 MB memory:  -> device: 0, name: Tesla M60, pci bus id: 0001:00:00.0, compute capability: 5.2


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 213, 64)           1229312   
                                                                 
 spatial_dropout1d (SpatialD  (None, 213, 64)          0         
 ropout1D)                                                       
                                                                 
 bidirectional (Bidirectiona  (None, 213, 256)         197632    
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 256)              394240    
 nal)                                                            
                                                                 
 dense (Dense)               (None, 19207)             4936199   
                                                        

In [10]:
model.fit(X[:int(len(X) / 2)], y[:int(len(X) / 2)], epochs=30, batch_size=128)

2023-01-15 06:23:29.197765: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 5241743956 exceeds 10% of free system memory.
2023-01-15 06:23:32.241512: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 5241743956 exceeds 10% of free system memory.
2023-01-15 06:23:58.336745: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8201


Epoch 1/30
534/534 [==============================] - 111s 113ms/step - loss: 3.7959 - accuracy: 0.3270
Epoch 2/30
534/534 [==============================] - 60s 113ms/step - loss: 2.6859 - accuracy: 0.4532
Epoch 3/30
534/534 [==============================] - 60s 113ms/step - loss: 2.2240 - accuracy: 0.5300
Epoch 4/30
534/534 [==============================] - 60s 113ms/step - loss: 1.9879 - accuracy: 0.5598
Epoch 5/30
534/534 [==============================] - 60s 113ms/step - loss: 1.8232 - accuracy: 0.5800
Epoch 6/30
534/534 [==============================] - 60s 113ms/st

In [14]:
model.save_weights('weights.h5')

In [15]:
model.fit(X[int(len(X) / 2):], y[int(len(X) / 2):], epochs=30, batch_size=128)

2023-01-15 06:56:27.448950: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 5241743956 exceeds 10% of free system memory.


Epoch 1/30
534/534 [==============================] - 60s 113ms/step - loss: 1.6779 - accuracy: 0.6485
Epoch 2/30
534/534 [==============================] - 60s 113ms/step - loss: 1.2639 - accuracy: 0.7033
Epoch 3/30
534/534 [==============================] - 60s 113ms/step - loss: 1.1351 - accuracy: 0.7241
Epoch 4/30
534/534 [==============================] - 60s 113ms/step - loss: 1.0490 - accuracy: 0.7362
Epoch 5/30
534/534 [==============================] - 60s 113ms/step - loss: 0.9939 - accuracy: 0.7455
Epoch 6/30
534/534 [==============================] - 60s 113ms/step - loss: 0.926

In [16]:
model.save_weights('weights.h5')

In [17]:
sequences = tokenizer.texts_to_sequences(finetune_df)
sequences = tf.keras.utils.pad_sequences(sequences)

In [18]:
X, y = sequences[:, :-1], sequences[:, -1]

In [19]:
y = tf.keras.utils.to_categorical(y, vocab_size - 1)

In [20]:
model.fit(X, y, epochs=80, batch_size=256)

Epoch 1/80
25/25 [==============================] - 4s 50ms/step - loss: 10.5217 - accuracy: 0.0727
Epoch 2/80
25/25 [==============================] - 1s 41ms/step - loss: 6.2430 - accuracy: 0.1465
Epoch 3/80
25/25 [==============================] - 1s 41ms/step - loss: 5.3349 - accuracy: 0.1776
Epoch 4/80
25/25 [==============================] - 1s 41ms/step - loss: 4.8780 - accuracy: 0.1990
Epoch 5/80
25/25 [==============================] - 1s 41ms/step - loss: 4.5211 - accuracy: 0.2208
Epoch 6/80
25/25 [==============================] - 1s 41ms/step - loss: 4.2253 - accuracy: 0.2350

In [21]:
model.save_weights('weights.h5')

In [22]:
def generate_word(model):
    start = np.random.randint(1, vocab_size, size = (1, 213))
    temp = np.argmax(model.predict(start))
    start = np.zeros((1, 213))
    start[0][-1] = temp
    word = idx2word[start[0][-1]]
    for i in range(20):
        new_idx = np.random.choice(range(1, vocab_size), p = model.predict(start).flatten())
        while new_idx == start[0][-1]:
            new_idx = np.random.choice(range(1, vocab_size), p = model.predict(start).flatten())
        word += ' ' + idx2word[new_idx]
        start[0][:-1] = start[0][1:]
        start[0][-1] = new_idx
    print(word)

In [25]:
generate_word(model)

1/1 [==============================] - 0s 28ms/step
starbucks aah accha ko karo jacket traffic ko jacket saala gaye ko sakta ko traffic konse ko kaisa ko kaisa add
